In [2]:
import cv2
import sys
import torch
import numpy as np
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from options.test_options import TestOptions

sys.argv = sys.argv[:1]

opt = TestOptions().parse() 

In [15]:
# 手动给 opt 赋值
opt.use_mask = True

opt.name = 'people'
opt.Arc_path = './checkpoints/arcface_model/arcface_checkpoint.tar'

In [6]:
import insightface
from insightface.app import FaceAnalysis

/data1/tides/new/miniconda3/envs/faceSwap/lib/python3.8/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
app = FaceAnalysis(name='antelopev2', root='./checkpoints')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

In [ ]:
netArc = torch.load(opt.Arc_path, map_location=torch.device("cpu"))
netArc = netArc.to(device)
netArc.eval()

In [64]:
from models.fs_model import fsModel

face_swap_model = fsModel()
face_swap_model.initialize(device, netArc, opt)

/home/tides/SimSwap/models/base_model.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  network.load_state_dict(torch.load(save_path))


In [31]:
source_image_path = 'demo_file/source_image/Iron_man.jpg'
target_video_path = 'demo_file/target_video/jirou_anni.mp4'
result_video_path = 'demo_file/result'
crop_size = 224

## Source图像预处理

In [12]:
source_image = cv2.imread(source_image_path)

In [13]:
source_image.shape

(1073, 769, 3)

In [32]:
det_result = app.get(source_image, crop_size)
det_result[0].keys()

/data1/tides/new/miniconda3/envs/faceSwap/lib/python3.8/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


dict_keys(['bbox', 'kps', 'det_score', 'landmark_3d_68', 'pose', 'landmark_2d_106', 'gender', 'age', 'embedding'])

In [21]:
from util import face_align_ffhqandnewarc as face_align

In [30]:
source_M, _ = face_align.estimate_norm(det_result[0]["kps"], crop_size) 
source_crop_align_image = cv2.warpAffine(source_image, source_M, (crop_size, crop_size), borderValue=0.0)

In [33]:
source_crop_align_image.shape

(224, 224, 3)

In [44]:
source_crop_align_image_pil = Image.fromarray(cv2.cvtColor(source_crop_align_image, cv2.COLOR_BGR2RGB)) 

In [45]:
transformer_Arcface = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

source_crop_align_image_pil = transformer_Arcface(source_crop_align_image_pil)
source_crop_align_image_pil = source_crop_align_image_pil.view(-1, source_crop_align_image_pil.shape[0], source_crop_align_image_pil.shape[1], source_crop_align_image_pil.shape[2])
source_crop_align_image_pil = source_crop_align_image_pil.to(device)

In [46]:
source_crop_align_image_pil

tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.0837, -2.0837, -2.0837],
          [-2.1179, -2.1179, -2.1179,  ..., -2.0837, -2.0837, -2.0837],
          [-2.1179, -2.1179, -2.1179,  ..., -2.0837, -2.0837, -2.0837],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.5604, -1.5256, -1.5256],
          [-1.8044, -1.8044, -

In [47]:
#create latent id
source_downsample = F.interpolate(source_crop_align_image_pil, size=(112,112))
latend_id = netArc(source_downsample)
latend_id = F.normalize(latend_id, p=2, dim=1)

In [48]:
latend_id.shape

torch.Size([1, 512])

## target视频预处理

In [55]:
target_video = cv2.VideoCapture(target_video_path)
target_video.set(cv2.CAP_PROP_POS_FRAMES, 150)

ret, frame = target_video.read()

In [56]:
target_detect_results = app.get(frame, crop_size)

In [57]:
target_M, _ = face_align.estimate_norm(target_detect_results[0]["kps"], crop_size) 
target_crop_align_image = cv2.warpAffine(frame, target_M, (crop_size, crop_size), borderValue=0.0)

In [61]:
def _totensor(array):
    tensor = torch.from_numpy(array)
    img = tensor.transpose(0, 1).transpose(0, 2).contiguous()
    return img.float().div(255)

In [62]:
target_crop_align_tenor = _totensor(cv2.cvtColor(target_crop_align_image, cv2.COLOR_BGR2RGB))[None,...].cuda()

In [63]:
target_crop_align_tenor.shape

torch.Size([1, 3, 224, 224])

In [68]:
swap_result = face_swap_model(None, target_crop_align_tenor, latend_id, None, True)

In [69]:
swap_result.shape

torch.Size([1, 3, 224, 224])

In [70]:
swap_result = swap_result.squeeze(0)  # 去掉 batch 维度
swap_result = swap_result.permute(1, 2, 0) 
swap_result = swap_result.cpu()
swap_result = (swap_result * 255).clamp(0, 255).byte().numpy()
cv2.imwrite('swap_result.jpg', swap_result)

True